1.  データ読み込み
2. IIR フィルター部分のネットワーク設計
3. 空間 (共線性が強すぎるからdropoutを強めに)と EEG ECoG 結合部分のネットワーク設計 EEG ECoG　を mean_squared_error に入力（同じ周波数フィルターが学習できるかも確認したいから別々に学習）
4. EEG->　時 -> 空　での学習と、EEG->空だけの学習
5. EEG を複製して入力すると、周波数フィルターを複数学習できるかテスト（そのままだと高周波は難しい気がする）

In [1]:
# -*- coding: utf_8 -*-  
import json
import pandas as pd
import numpy as np
import datetime
import math
import time

import chainer
from chainer import cuda, Function, gradient_check, utils, Variable
from chainer import optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L



import pickle
import sys
import random
import h5py

In [2]:
test=h5py.File('Ma/DeepANES.mat')

ECoG = np.array(test['ECoG'])
EEG = np.array(test['EEG'])

In [3]:
EEG = EEG.astype('float32')
ECoG = ECoG.astype('float32')

In [4]:
class BasicSpacePerceptron(Chain):
    def __init__(self):
        super(BasicSpacePerceptron, self).__init__(
            transformEEG = L.Linear(17, 256)# EEG 空間
        )
        
    def __call__(self, EEG, ECoG, train=False):
        eeg_space = self.transformEEG(EEG)
        eeg_space_dropout = F.dropout(eeg_space, train=train, ratio=0.5)
        ecog  = ECoG
        y = eeg_space_dropout - ecog
        return y
   

In [5]:
basicSpacePerceptron = BasicSpacePerceptron()
optimizer = optimizers.AdaDelta()
optimizer.setup(basicSpacePerceptron)
optimizer.weight_decay(0.0005)


In [18]:
basicSpacePerceptron.zerograds()
loop = 200
cor_history = np.zeros((1, 256))
loss_history = np.zeros((loop, 1))
st = time.time()
for i in range(loop):
    loss = F.sum(basicSpacePerceptron(Variable(EEG), Variable(ECoG), train=True) **2)
    loss.backward()
    optimizer.update()
    loss_history[i] = loss.data

IndexError: index 1 is out of bounds for axis 0 with size 1

In [20]:
test = EEG.dot(basicSpacePerceptron.transformEEG.W.data.T)
for j in range(256):
    cor_history[0,  j] = np.corrcoef(test[j,:], ECoG[j,:])[0, 1]
print(loss.data)
print(time.time()-st)

1415462038536192.0
806.8021790981293


### 時間フィルター FIR と IIR を試す

In [ ]:
# y = [y0, y1, y2, y3] 4 order までしかやらん！

y0 = model.IIR_linear()

In [163]:
class TimeSpacePerceptron(Chain):
    def __init__(self):
        tt = 299997
        super(TimeSpacePerceptron, self).__init__(
            transformEEG = L.Linear(17, 256)# EEG 空間
            )
        self.IIR_EEG = [L.Linear(7, 1)] * 17, #copy されているかは要チェック
        self.IIR_ECoG = [L.Linear(7, 1)] * 256
        self.EEG_IIRs_prev = [[Variable(np.zeros((tt, 1)).astype('float32'))]*4]*17
        self.ECoG_IIRs_prev = [[Variable(np.zeros((tt, 1)).astype('float32'))]*4]*256
        
    def __call__(self, EEG_t, ECoG_t, train=False):
        EEG_IIRs_prev = self.EEG_IIRs_prev
        ECoG_IIRs_prev = self.ECoG_IIRs_prev
        # EEG_t は 16 * time * time_delay
        EEG_IIRs = [] # ネットワーク y0, y1, y2, y3 が入る ch の数だけ配列
        for ch in range(EEG_t.data.shape[1]):
            EEG_IIRs.append(self.IIR_learn(EEG_t[:, ch, :], EEG_IIRs_prev[ch], self.IIR_EEG[0][ch]))
        ECoG_IIRs = [] # ネットワーク y0, y1, y2, y3 が入る ch の数だけ配列
        for ch in range(ECoG_t.data.shape[1]):
            ECoG_IIRs.append(self.IIR_learn(ECoG_t[:, ch, :], ECoG_IIRs_prev[ch], self.IIR_ECoG[ch]))
        EEG_filtered, ECoG_filtered = [], []
        for ch in range(EEG_t.data.shape[1]):
            EEG_filtered.append(EEG_IIRs[ch][0])
        EEG_filtered = F.concat(tuple(EEG_filtered))
        for ch in range(ECoG_t.data.shape[1]):
            ECoG_filtered.append(ECoG_IIRs[ch][0])
        ECoG_filtered = F.concat(tuple(ECoG_filtered))
        # ここまで IIR
        eeg_space = self.transformEEG(EEG_filtered)
        eeg_space_dropout = F.dropout(eeg_space, train=train, ratio=0.5)
        ecog  = ECoG_filtered
        y = eeg_space_dropout - ecog
        self.EEG_IIRs_prev = EEG_IIRs
        self.ECoG_IIRs_prev = ECoG_IIRs
        return y
    
    def IIR_learn(self, data, IIRs_prev, IIR_transform):
        #data = Variable(data.reshape((data.shape[1], data.shape[2])))
        x = F.concat((data, IIRs_prev[0], IIRs_prev[1], IIRs_prev[2]))
        y= IIR_transform(x)
        return([y, IIRs_prev[0], IIRs_prev[1], IIRs_prev[2]])
    

In [164]:
timeSpacePerceptron = TimeSpacePerceptron()
optimizer = optimizers.AdaDelta()
optimizer.setup(basicSpacePerceptron)
optimizer.weight_decay(0.0005)


In [ ]:
timeSpacePerceptron.zerograds()
loop = 3
cor_history = np.zeros((loop, 256))
loss_history = np.zeros((loop, 1))
st = time.time()
EEG_t = np.zeros((EEG.shape[0]-3, EEG.shape[1], 4))
ECoG_t = np.zeros((ECoG.shape[0]-3, ECoG.shape[1], 4))
for t in range(4):
    for ch in range(EEG.shape[1]):
        EEG_t[:,ch,t] = EEG[t:EEG.shape[0]-3+t, ch]
    for ch in range(ECoG.shape[1]):
        ECoG_t[:,ch,t] = ECoG[t:ECoG.shape[0]-3+t, ch]
EEG_t = Variable(EEG_t.astype('float32'))
ECoG_t = Variable(ECoG_t.astype('float32'))
for i in range(loop):
    loss = F.sum(timeSpacePerceptron(EEG_t, ECoG_t, train=True) **2)
    loss.backward()
    optimizer.update()
    loss_history[i] = loss.data
    test = EEG.dot(timeSpacePerceptron.transformEEG.W.data.T)
    for j in range(256):
        cor_history[i, j] = np.corrcoef(test[j,:], ECoG[j,:])[0, 1]
    print(loss.data)
print(time.time()-st)

In [104]:
np.corrcoef(test[0,:], ECoG[0,:])

array([[ 1.        ,  0.00596444],
       [ 0.00596444,  1.        ]])

In [ ]:
np.corrcoef(test, ECoG).shape

In [79]:
EEG_t.shape

(299997, 17, 4)

In [124]:
a = Variable(np.zeros(2))

In [139]:
timeSpacePerceptron.IIR_EEG[1][0]

IndexError: tuple index out of range